In [ ]:
import torch
import tvm
from tvm import relay
from tvm.relay import save_param_dict
from tvm.contrib import graph_executor
import os
import torchvision.models as models
squeezenet = models.squeezenet1_0(weights=models.SqueezeNet1_0_Weights.IMAGENET1K_V1).eval()


# === Trace ===
x = torch.randn(1, 3, 224, 224)
scripted = torch.jit.trace(squeezenet, x).eval()

# === Convert ===
mod, params = relay.frontend.from_pytorch(scripted, [("input0", x.shape)])
mod = relay.transform.InferType()(mod)  # ensure types

# === Compile ===
target = "llvm"
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

# === Export ===
outdir = "/workspaces/TVM-Prep-guide/tvm_cpp/artifacts/SqueezeNet"
os.makedirs(outdir, exist_ok=True)

lib.export_library(f"{outdir}/SqueezeNet_tvm.so")

with open(f"{outdir}/SqueezeNet_tvm.json", "w") as f:
    f.write(lib.get_graph_json())

# *** CRITICAL FIX HERE ***
with open(f"{outdir}/SqueezeNet_tvm.params", "wb") as f:
    # Use lib.get_params() instead of the original 'params' variable
    f.write(save_param_dict(lib.get_params()))